# Find the differences between the DTU10 and EGM08 geoids
****

#### Intall rioxarray in Google Colab

In [ ]:
pip install rioxarray

#### Download the data

In [ ]:
!wget https://s3-eu-west-1.amazonaws.com/download.agisoft.com/gtg/us_nga_egm2008_1.tif #EGM08
!wget https://ftp.space.dtu.dk/pub/DTU10/1_MIN/DTU10MDT_1min.nc #DTU10 (MDT)
!wget https://ftp.space.dtu.dk/pub/DTU10/1_MIN/DTU10MSS_1min.nc #DTU10 (MSS)

#### Import the libraries to be used in the analysis

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray

### Step 1: Import the data
****

#### Load the DTU10MSS (mean sea surface) and the DTU10MDT (mean dynamic topography)

In [ ]:
dtu10mss = xr.open_dataarray("./DTU10MSS_1min.nc")
dtu10mss

In [ ]:
dtu10mdt = xr.open_dataarray("./DTU10MDT_1min.nc")
dtu10mdt

#### Load the EGM08 geoid, change the coords names and drop the band coordinate

In [ ]:
egm08 = rioxarray.open_rasterio("./us_nga_egm2008_1.tif").rename("egm08").rename({"x": "lon", "y": "lat"}).squeeze().drop_vars("band")
egm08

### Step 2: Adjust the DTU10 coordinates and calculate the EDT10 geoid
****

#### Change the longitude from 0-360 to -180-180

In [ ]:
dtu10mss["lon"] = (dtu10mss["lon"] + 180) % 360 - 180
dtu10mdt["lon"] = (dtu10mdt["lon"] + 180) % 360 - 180

#### Sort the longitude

In [ ]:
dtu10mss = dtu10mss.sortby("lon")
dtu10mdt = dtu10mdt.sortby("lon")

#### Calculate the geoid

In [ ]:
dtu10 = dtu10mss - dtu10mdt

### Step 3: Adjust the EGM08 coordinates
****

#### Sort the latitude and resample the dataset to the DTU10 coordinates

In [ ]:
egm08 = egm08.sortby("lat")
egm08_fixed = egm08.interp(lon=dtu10["lon"], lat=dtu10["lat"])

### Step 4: Calculate the differences between geoids
****

### Calculate the differences

In [ ]:
geoid_diff = abs(dtu10) - abs(egm08_fixed)

### Step 5: Plot the data and save to an file
****

### Plot DTU geoid

In [ ]:
### Subset
dtu10plot = dtu10.sel(lon=slice(-92.6, -92.3), lat=slice(19, 19.19))
### Plot
fig = plt.figure(figsize=(6, 4), dpi=300)
ax = fig.subplots()
mesh = ax.pcolormesh(dtu10plot["lon"], dtu10plot["lat"], dtu10plot)
contour = ax.contour(dtu10plot["lon"], dtu10plot["lat"], dtu10plot, colors= "white")
fig.colorbar(mesh, ax=ax)
ax.clabel(contour, inline=True, fontsize=7)
ax.set_title("DTU10 (m)")
plt.tight_layout()

### Plot EGM08 geoid

In [ ]:
### Subset
egm08plot = egm08.sel(lon=slice(-92.6, -92.3), lat=slice(19, 19.19))
### Plot
fig = plt.figure(figsize=(6, 4), dpi=300)
ax = fig.subplots()
mesh = ax.pcolormesh(egm08plot["lon"], egm08plot["lat"], egm08plot)
contour = ax.contour(egm08plot["lon"], egm08plot["lat"], egm08plot, colors= "white")
fig.colorbar(mesh, ax=ax)
ax.clabel(contour, inline=True, fontsize=7)
ax.set_title("EGM08 (m)")
plt.tight_layout()

### Plot diff 

In [ ]:
### Subset
geoid_diff_cut = geoid_diff.sel(lon=slice(-92.6, -92.3), lat=slice(19, 19.19))
### Plot
fig = plt.figure(figsize=(6, 4), dpi=300)
ax = fig.subplots()
mesh = ax.pcolormesh(geoid_diff_cut["lon"], geoid_diff_cut["lat"], geoid_diff_cut)
contour = ax.contour(geoid_diff_cut["lon"], geoid_diff_cut["lat"], geoid_diff_cut, colors= "white")
fig.colorbar(mesh, ax=ax)
ax.clabel(contour, inline=True, fontsize=7)
ax.set_title("DTU10 - EGM08 (m)")
plt.tight_layout()

#### Export the data

In [ ]:
geotiff = geoid_diff.sel(lon=slice(-98, -81), lat=slice(17.5, 30)).assign_coords({"spatial_ref": egm08["spatial_ref"]}).rename({"lon": "x", "lat":"y"})
geotiff.rio.to_raster("./diff.tif")